In [1]:
from tools.torch_lib import *

In [2]:
### uncomment if libs are not installed:
# %pip install numpy
# %pip install pandas
# pytorch without gpu support:
# %pip install torch torchvision torchaudio
## tf without gpu support
# pip install tensorflow
# pip install keras
# pip install onnx
# pip install onnx2keras

In [3]:
models_dir = "models/"
model_name = "model.pt"
model_path = models_dir + model_name
scripted_model = torch.jit.load(model_path)

### convert to onnx

In [4]:
print(scripted_model)

RecursiveScriptModule(
  original_name=LinearModel
  (linears): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=Linear)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=Linear)
    (4): RecursiveScriptModule(original_name=Linear)
    (5): RecursiveScriptModule(original_name=Linear)
    (6): RecursiveScriptModule(original_name=Linear)
  )
  (activations): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(original_name=ReLU)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=ReLU)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=ReLU)
    (5): RecursiveScriptModule(original_name=ReLU)
    (6): RecursiveScriptModule(original_name=ReLU)
  )
)


In [5]:
input_tensor = torch.randn(1, 3)
scripted_model(input_tensor)

tensor([[0.0981]], grad_fn=<ReluBackward0>)

In [6]:
input_tensor

tensor([[-0.2350,  1.2979,  0.8272]])

In [7]:
### define model class for proper onnx export
class LinearModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        module_list = []
        for i in range(layers_count - 1):
            module_list.append(nn.Linear(layers_dims[i], layers_dims[i + 1]))
        module_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(module_list)
        self.activations = nn.ModuleList(activations_list)

    def forward(self, x):
        y = x

        for lin, act in zip(self.linears, self.activations):
            y = lin(y)
            y = act(y)

        return y

In [8]:
output_len = 1
input_len = 3
layers_dims = [input_len, 30, 100, 700, 100, 30, output_len]

layers_count = len(layers_dims)
activations_string_list = ['relu' for i in range(layers_count)]

# convert scripted model to model class object
model_object = LinearModel(layers_dims, activations_string_list, output_len)
model_object.load_state_dict(scripted_model.state_dict())
print(model_object)

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=30, bias=True)
    (1): Linear(in_features=30, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=700, bias=True)
    (3): Linear(in_features=700, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=30, bias=True)
    (5): Linear(in_features=30, out_features=1, bias=True)
    (6): Linear(in_features=1, out_features=1, bias=True)
  )
  (activations): ModuleList(
    (0-6): 7 x ReLU()
  )
)


In [9]:
model_object(input_tensor)

tensor([[0.0981]], grad_fn=<ReluBackward0>)

In [10]:
torch.onnx.export(model_object, input_tensor, models_dir + "model.onnx", export_params=True, opset_version=11)
#onnx_binary.save(models_dir + "onnx_model.onnx")

### Convert onnx model to keras

In [11]:
import onnx
from onnx2keras import onnx_to_keras

onnx_model = onnx.load(models_dir + "model.onnx")
keras_model = onnx_to_keras(onnx_model, ['onnx::Gemm_0'], name_policy='renumerate')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> onnx::Gemm_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 28.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight linears.0.weight with shape (30, 3).
DEBUG:onnx2keras:Found weight linears.0.bias with shape (30,).
DEBUG:onnx2keras:Found weight linears.1.weight with shape (100, 30).
DEBUG:onnx2keras:Found weight linears.1.bias with shape (100,).
DEBUG:onnx2keras:Found weight linears.2.weight with shape (700, 100).
DEBUG:onnx2keras:Found weight linears.2.bias with shape (700,).
DEBUG:onnx2keras:Found weight linears.3.weight with shape (100, 700).
DEBUG:onnx2keras:Found weight linears.3.bias with shape (100,).
DEBUG:onnx2keras:Found weight linears.4.weight with shape (30, 100).
DEBUG:onnx2keras:Found weight linears.4.bias with shape (30,).
DEBUG:onnx2keras:Found weight linears.5.weight with shap


DEBUG:onnx2keras:Found input onnx::Gemm_0 with shape [3]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Gemm
DEBUG:onnx2keras:node_name: /linears.0/Gemm_output_0
DEBUG:onnx2keras:node_params: {'alpha': 1.0, 'beta': 1.0, 'transB': 1, 'change_ordering': False, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name onnx::Gemm_0).
DEBUG:onnx2keras:Check input 1 (name linears.0.weight).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name linears.0.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:gemm:Convert GEMM with bias.
DEBUG:onnx2keras:gemm:Transposing W matrix.
DEBUG:onnx2keras:gemm:Input units 3

In [12]:
keras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 onnx::Gemm_0 (InputLayer)   [(None, 3)]               0         
                                                                 
 LAYER_0 (Dense)             (None, 30)                120       
                                                                 
 LAYER_1 (Activation)        (None, 30)                0         
                                                                 
 LAYER_2 (Dense)             (None, 100)               3100      
                                                                 
 LAYER_3 (Activation)        (None, 100)               0         
                                                                 
 LAYER_4 (Dense)             (None, 700)               70700     
                                                                 
 LAYER_5 (Activation)        (None, 700)               0     

In [13]:
keras_model.predict(input_tensor.numpy())

1/1 [==============================] - 0s 171ms/step


array([[0.09807725]], dtype=float32)

In [14]:
# save keras model
keras_model.save(models_dir + "keras_model.h5")

C:\GitHub\NSU_Graduate\PtKerasConversion\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
DEBUG:h5py._conv:Creating converter from 5 to 3


In [15]:
from keras.models import load_model
# check loaded keras model
keras_model = load_model(models_dir + "keras_model.h5")
keras_model.predict(input_tensor.numpy())

DEBUG:h5py._conv:Creating converter from 3 to 5


1/1 [==============================] - 0s 72ms/step


array([[0.09807725]], dtype=float32)

### convert model to keras2cpp lib format and save it
keras2cpp lib doesn't support Input() layer so convert model to contain only
Dense & Activation layers

In [16]:
from keras import Sequential
from keras.layers import Dense, Activation

input_dim = 3
converted_model = Sequential()

for layer in keras_model.layers[1:]:
    if isinstance(layer, Dense):
        insert_dense_layer = Dense(units=layer.units, input_shape=(input_dim,))
        converted_model.add(insert_dense_layer)
        converted_model.layers[-1].set_weights(layer.get_weights())
        input_dim = (layer.units, )
    elif isinstance(layer, Activation):
        insert_activation_layer = Activation(activation=layer.activation)
        converted_model.add(insert_activation_layer)
    else:
        assert False and "invalid layer"

converted_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                120       
                                                                 
 activation (Activation)     (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 100)               3100      
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 700)               70700     
                                                                 
 activation_2 (Activation)   (None, 700)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               7

In [17]:
# check converted model inference
converted_model.predict(input_tensor.numpy())

1/1 [==============================] - 0s 99ms/step


array([[0.09807725]], dtype=float32)

In [18]:
from tools.keras2cpp import export_model
export_model(converted_model, models_dir + 'keras2cpp.model')